In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install module

In [174]:
from deepagri.data_full import get_df_full

from tensorflow.keras import models,layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



In [142]:
df=get_df_full(agg_type='W')

/Users/constantintalandier/code/Constantier/deepagri/deepagri/meteo_agg.py:99: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  df_agg.loc[:, idx[:,cols_to_shift]] = (df_agg.loc[:, idx[:,cols_to_shift]]
/Users/constantintalandier/code/Constantier/deepagri/deepagri/meteo_agg.py:99: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  df_agg.loc[:, idx[:,cols_to_shift]] = (df_agg.loc[:, idx[:,cols_to_shift]]


In [143]:
df

,Agriculteur,Production,RatioSep,RatioOct,Prix_matos,tmax_c 01,tmax_c 02,tmax_c 03,tmax_c 04,tmax_c 05,...,sunhour 43_n-1,sunhour 44_n-1,sunhour 45_n-1,sunhour 46_n-1,sunhour 47_n-1,sunhour 48_n-1,sunhour 49_n-1,sunhour 50_n-1,sunhour 51_n-1,sunhour 52_n-1
2010-1,2820.041792,239640.3,1.229426,1.255319,92.741667,2.0,1.0,1.0,1.0,1.0,...,7.328571,8.700000,6.328571,6.057143,8.214286,6.714286,5.528571,5.257143,6.428571,6.742857
2010-2,3563.698886,1485134.7,1.229426,1.255319,92.741667,2.0,1.0,2.0,0.0,1.0,...,6.142857,7.514286,3.657143,6.514286,6.842857,3.671429,3.900000,5.185714,6.914286,4.728571
2010-3,4276.654443,307450.0,1.229426,1.255319,92.741667,0.0,1.0,1.0,2.0,1.0,...,6.871429,8.700000,5.614286,7.014286,7.728571,6.157143,5.614286,4.414286,5.342857,6.642857
2010-4,1462.803043,6233.7,1.229426,1.255319,92.741667,1.0,2.0,1.0,1.0,1.0,...,7.600000,8.700000,8.471429,8.700000,7.971429,7.942857,7.700000,8.185714,8.342857,6.142857
2010-5,1406.337173,9753.9,1.229426,1.255319,92.741667,1.0,0.0,1.0,1.0,1.0,...,8.471429,8.700000,6.342857,8.457143,7.728571,7.700000,7.185714,8.085714,8.000000,6.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-91,422.282609,243243.0,1.104197,1.072703,89.891667,1.0,1.0,2.0,1.0,1.0,...,5.457143,5.400000,7.500000,6.028571,7.942857,7.800000,4.871429,3.371429,6.542857,3.914286
2021-93,80.734902,2739.0,1.104197,1.072703,89.891667,4.0,2.0,6.0,4.0,2.0,...,7.800000,7.542857,7.528571,7.514286,8.700000,7.728571,7.000000,6.485714,4.914286,7.442857
2021-94,65.237675,3483.0,1.104197,1.072703,89.891667,2.0,0.0,2.0,1.0,1.0,...,5.942857,5.400000,7.742857,6.257143,7.185714,7.285714,4.871429,3.357143,6.800000,3.900000
2021-95,213.709486,198121.0,1.104197,1.072703,89.891667,3.0,0.0,2.0,1.0,1.0,...,5.685714,5.171429,6.785714,5.271429,6.414286,7.285714,4.914286,4.171429,5.842857,3.100000


In [166]:
X_test=df.iloc[-93:].drop(columns=['Production'])
#X_test=df.iloc[-93:][['Surface_n-1','Agriculteur','windspeed_max sept-jan_n-1']]
y_test=df.iloc[-93:]['Production']
X_train=df.iloc[:-93].drop(columns=['Production'])
#X_train=df.iloc[:-93][['Surface_n-1','Agriculteur','windspeed_max sept-jan_n-1']]
y_train=df.iloc[:-93]['Production']

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')


In [167]:
df.shape

(1116, 423)

In [168]:
#instensiation du model
model=models.Sequential()
#Création des layers

model.add(layers.Dense(256,activation='relu',input_dim=422)) # Layer avec 10 neurones, pas besoin de input_dim 
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128,activation='relu')) # Layer avec 10 neurones, pas besoin de input_dim 
model.add(layers.Dense(64,activation='relu')) # Layer avec 10 neurones, pas besoin de input_dim 
model.add(layers.Dense(32,activation='relu')) # Layer avec 10 neurones, pas besoin de input_dim 
model.add(layers.Dense(16,activation='relu')) # Layer avec 10 neurones, pas besoin de input_dim 
model.add(layers.Dense(1,activation='linear')) # Layer de sortie pour une classification (1 neurone, 'sigmoid')
#Compilation du model avec les fonctions d'optimisation et de loss

In [175]:
model.compile(loss='mape', # We've already mentioned this loss function in Logistic Regression
                  optimizer=Adam(learning_rate=0.0001), # Optimizer in Deep Learning = solver in Machine Learning | Adam = our best friend
                  metrics=['mae']) # Let's focus on the accuracy, our dataset is balanced
es = EarlyStopping(patience=50,restore_best_weights=True) #Attend 30 epochs ou la loss du val set remonte pour stopper

In [176]:
model.fit(X_train,y_train,batch_size=32,epochs=1000,validation_data=(X_test,y_test),callbacks=[es])

Epoch 1/1000
32/32 [==============================] - 0s 3ms/step - loss: 51.7058 - mae: 170241.5781 - val_loss: 253.3636 - val_mae: 168618.3594
Epoch 2/1000
32/32 [==============================] - 0s 2ms/step - loss: 50.3329 - mae: 166055.9375 - val_loss: 294.0043 - val_mae: 166258.0469
Epoch 3/1000
32/32 [==============================] - 0s 2ms/step - loss: 50.5197 - mae: 165059.9219 - val_loss: 240.6493 - val_mae: 172211.1562
Epoch 4/1000
32/32 [==============================] - 0s 2ms/step - loss: 50.3521 - mae: 165919.3438 - val_loss: 266.0240 - val_mae: 169101.5312
Epoch 5/1000
32/32 [==============================] - 0s 2ms/step - loss: 49.9123 - mae: 165613.9531 - val_loss: 314.4067 - val_mae: 169862.9844
Epoch 6/1000
32/32 [==============================] - 0s 2ms/step - loss: 51.1293 - mae: 170435.5469 - val_loss: 249.9830 - val_mae: 172517.7031
Epoch 7/1000
32/32 [==============================] - 0s 2ms/step - loss: 50.4807 - mae: 165689.3906 - val_loss: 225.1738 - val_ma

32/32 [==============================] - 0s 2ms/step - loss: 49.0656 - mae: 166671.2344 - val_loss: 268.0763 - val_mae: 167046.6875
Epoch 58/1000
32/32 [==============================] - 0s 2ms/step - loss: 49.8536 - mae: 167037.6094 - val_loss: 236.2997 - val_mae: 167110.6875
Epoch 59/1000
32/32 [==============================] - 0s 2ms/step - loss: 50.0397 - mae: 166901.0156 - val_loss: 230.3234 - val_mae: 169364.2031
Epoch 60/1000
32/32 [==============================] - 0s 2ms/step - loss: 50.6622 - mae: 170944.1406 - val_loss: 279.0197 - val_mae: 169528.2656
Epoch 61/1000
32/32 [==============================] - 0s 2ms/step - loss: 48.6969 - mae: 166381.7500 - val_loss: 279.8461 - val_mae: 167687.2969
Epoch 62/1000
32/32 [==============================] - 0s 2ms/step - loss: 50.4854 - mae: 168317.6562 - val_loss: 215.4771 - val_mae: 171371.5625
Epoch 63/1000
32/32 [==============================] - 0s 2ms/step - loss: 50.0331 - mae: 168914.5781 - val_loss: 268.6945 - val_mae: 1711

In [16]:
X_train.shape

(1023, 27)

In [26]:
X_test = np.asarray(X_test).astype('float32')


In [27]:
y_test = np.asarray(y_test).astype('float32')

In [33]:
model

In [109]:
from deepagri.data_pred import get_X_pred

In [110]:
df_pred=get_X_pred()

In [111]:
df_pred

,tmax_c 01,tmax_c 02,tmax_c 03,tmax_c 04,tmax_c 05,tmax_c 06,tmax_c 07,tmax_c 08,tmax_c 10_n-1,tmax_c 11_n-1,...,sunhour 17_n-1,sunhour 18_n-1,sunhour 19_n-1,sunhour 20_n-1,sunhour 21_n-1,sunhour 22_n-1,Agriculteur,RatioSep,RatioOct,Prix_matos
2022-1,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,9.914286,9.514286,12.914286,11.271429,13.671429,14.500000,1679.279614,1.091368,1.098,91.6
2022-10,1.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,...,13.400000,10.128571,11.700000,13.057143,13.842857,13.842857,2133.606402,1.091368,1.098,91.6
2022-11,2.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,...,11.042857,10.671429,12.414286,9.842857,13.628571,14.500000,2209.221134,1.091368,1.098,91.6
2022-12,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,10.300000,9.314286,13.057143,10.028571,13.771429,14.500000,4947.604107,1.091368,1.098,91.6
2022-13,2.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0,2.0,0.0,...,11.657143,12.685714,13.071429,11.657143,14.042857,13.928571,1857.125823,1.091368,1.098,91.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-90,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,12.500000,10.042857,11.842857,12.214286,13.285714,14.500000,73.788254,1.091368,1.098,91.6
2022-91,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,13.657143,11.328571,12.000000,12.485714,14.085714,14.500000,411.009300,1.091368,1.098,91.6
2022-93,4.0,2.0,4.0,2.0,2.0,4.0,2.0,2.0,4.0,2.0,...,11.657143,13.471429,13.357143,11.185714,14.028571,13.057143,84.097128,1.091368,1.098,91.6
2022-94,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,13.657143,10.871429,12.142857,12.485714,13.942857,14.500000,66.780567,1.091368,1.098,91.6


In [62]:
from deepagri.data_pop import get_data_population
from deepagri.data_price import get_prices
from deepagri.data_prod import get_production
from deepagri.meteo_agg import agg_meteo
import os
import pandas as pd

In [60]:
pwd

'/Users/constantintalandier/code/Constantier/deepagri/notebooks'

In [128]:
'''
Return a dataframe with all the data of meteo of the year and the year before,
prices ratios, population, yield n-1 and price of the matos.
'''
PATH=os.path.dirname('/Users/constantintalandier/code/Constantier/deepagri/notebooks')
df_pop=get_data_population()
df_prices=get_prices()
df_prod=get_production()
df_meteo=agg_meteo(agg_type='W')
# df_rendement=get_data_rendement()
df_matos=pd.read_csv(os.path.join(os.path.join(PATH,'raw_data'),'matos.csv'),index_col='0')

/Users/constantintalandier/code/Constantier/deepagri/deepagri/meteo_agg.py:99: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  df_agg.loc[:, idx[:,cols_to_shift]] = (df_agg.loc[:, idx[:,cols_to_shift]]
/Users/constantintalandier/code/Constantier/deepagri/deepagri/meteo_agg.py:99: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  df_agg.loc[:, idx[:,cols_to_shift]] = (df_agg.loc[:, idx[:,cols_to_shift]]


In [129]:
print(df_pop.shape,df_prices.shape,df_meteo.shape,df_prod.shape,df_matos.shape,df_pred.shape)

(1209, 1) (1209, 2) (1302, 429) (1209, 1) (1209, 1) (93, 235)


In [130]:
df_meteo_copy=df_meteo.copy()

In [131]:
df_meteo_copy.columns=[' '.join(col).strip() for col in df_meteo_copy.columns.values] #Suppression du multiindex des columns
df_meteo_copy

,tmax_c 01,tmax_c 02,tmax_c 03,tmax_c 04,tmax_c 05,tmax_c 06,tmax_c 07,tmax_c 08,tmax_c 09,tmax_c 10,...,sunhour 43_n-1,sunhour 44_n-1,sunhour 45_n-1,sunhour 46_n-1,sunhour 47_n-1,sunhour 48_n-1,sunhour 49_n-1,sunhour 50_n-1,sunhour 51_n-1,sunhour 52_n-1
date_dep,,,,,,,,,,,,,,,,,,,,,
2009-1,0.0,3.0,0.0,2.0,1.0,1.0,2.0,2.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-11,0.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-13,1.0,1.0,0.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-90,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,...,7.300000,4.885714,6.728571,6.685714,5.128571,3.942857,3.914286,7.871429,5.600000,5.114286
2022-91,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,...,7.057143,7.028571,7.242857,5.685714,6.871429,5.228571,5.771429,8.000000,6.085714,5.014286
2022-93,4.0,2.0,4.0,2.0,2.0,4.0,2.0,2.0,2.0,NaN,...,6.600000,6.600000,7.028571,7.742857,6.028571,8.214286,7.485714,8.700000,7.185714,7.428571


In [132]:
df_meteo_copy.columns = df_meteo_copy.columns.get_level_values(0)
df_meteo_copy

,tmax_c 01,tmax_c 02,tmax_c 03,tmax_c 04,tmax_c 05,tmax_c 06,tmax_c 07,tmax_c 08,tmax_c 09,tmax_c 10,...,sunhour 43_n-1,sunhour 44_n-1,sunhour 45_n-1,sunhour 46_n-1,sunhour 47_n-1,sunhour 48_n-1,sunhour 49_n-1,sunhour 50_n-1,sunhour 51_n-1,sunhour 52_n-1
date_dep,,,,,,,,,,,,,,,,,,,,,
2009-1,0.0,3.0,0.0,2.0,1.0,1.0,2.0,2.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-11,0.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-13,1.0,1.0,0.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-90,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,...,7.300000,4.885714,6.728571,6.685714,5.128571,3.942857,3.914286,7.871429,5.600000,5.114286
2022-91,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,...,7.057143,7.028571,7.242857,5.685714,6.871429,5.228571,5.771429,8.000000,6.085714,5.014286
2022-93,4.0,2.0,4.0,2.0,2.0,4.0,2.0,2.0,2.0,NaN,...,6.600000,6.600000,7.028571,7.742857,6.028571,8.214286,7.485714,8.700000,7.185714,7.428571


In [140]:
df_meteo_copy[['tmax_c 52_n-1']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 2009-1 to 2022-95
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tmax_c 52_n-1  1209 non-null   float64
dtypes: float64(1)
memory usage: 20.3+ KB


In [107]:
df_meteo.dropna()

tmax_c                                                   ...  \
week_of_year     01   02   03   04   05   06   07   08 10_n-1 11_n-1  ...   
date_dep                                                              ...   
2010-1          2.0  1.0  1.0  1.0  1.0  1.0  2.0  2.0    0.0    1.0  ...   
2010-10         2.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0    2.0    2.0  ...   
2010-11         1.0  1.0  1.0  3.0  1.0  1.0  2.0  1.0    2.0    1.0  ...   
2010-12         1.0  2.0  1.0  2.0  1.0  1.0  1.0  1.0    1.0    1.0  ...   
2010-13         1.0  1.0  1.0  2.0  2.0  2.0  2.0  1.0    1.0    2.0  ...   
...             ...  ...  ...  ...  ...  ...  ...  ...    ...    ...  ...   
2022-90         1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0    0.0    1.0  ...   
2022-91         2.0  1.0  2.0  1.0  1.0  1.0  1.0  1.0    1.0    2.0  ...   
2022-93         4.0  2.0  4.0  2.0  2.0  4.0  2.0  2.0    4.0    6.0  ...   
2022-94         1.0  1.0  2.0  1.0  1.0  1.0  1.0  1.0    2.0    2.0  ...   
2022-95         2.0  1.0  1.0  1.0  0.0  1.0  2.0  1.0    1.0    2.0  ...   

               sunhour                                                    \
week_of_year    43_n-1    44_n-1    45_n-1    46_n-1    47_n-1    48_n-1   
date_dep                                                                   
2010-1        7.328571  8.700000  6.328571  6.057143  8.214286  6.714286   
2010-10       6.857143  7.985714  4.642857  6.757143  6.971429  4.514286   
2010-11       6.928571  8.700000  6.357143  7.271429  8.700000  8.700000   
2010-12       5.957143  8.700000  4.228571  7.500000  8.457143  6.471429   
2010-13       6.914286  8.014286  8.228571  8.228571  7.728571  8.457143   
...                ...       ...       ...       ...       ...       ...   
2022-90       7.300000  4.885714  6.728571  6.685714  5.128571  3.942857   
2022-91       7.057143  7.028571  7.242857  5.685714  6.871429  5.228571   
2022-93       6.600000  6.600000  7.028571  7.742857  6.028571  8.214286   
2022-94       7.042857  6.557143  7.242857  5.671429  6.614286  4.957143   
2022-95       6.814286  6.542857  7.471429  5.157143  6.357143  5.200000   

                                                      
week_of_year    49_n-1    50_n-1    51_n-1    52_n-1  
date_dep                                              
2010-1        5.528571  5.257143  6.428571  6.742857  
2010-10       3.857143  4.914286  5.242857  5.428571  
2010-11       7.171429  7.685714  6.928571  7.671429  
2010-12       5.914286  7.185714  6.642857  6.128571  
2010-13       7.728571  7.957143  8.700000  5.414286  
...                ...       ...       ...       ...  
2022-90       3.914286  7.871429  5.600000  5.114286  
2022-91       5.771429  8.000000  6.085714  5.014286  
2022-93       7.485714  8.700000  7.185714  7.428571  
2022-94       5.771429  8.000000  5.600000  5.014286  
2022-95       5.471429  7.042857  5.485714  4.757143  

[1209 rows x 561 columns]

In [108]:

df=df_pop.merge(df_prod,left_index=True,right_index=True)
df=df.merge(df_prices,left_index=True,right_index=True)
df=df.merge(df_matos,left_index=True,right_index=True)
df=df.rename(columns={'1':"Prix_matos"})
# df=df.merge(df_rendement,left_index=True,right_index=True)

df_meteo.columns=[' '.join(col).strip() for col in df_meteo.columns.values] #Suppression du multiindex des columns
df_meteo.columns = df_meteo.columns.get_level_values(0)

df_meteo=df_meteo.loc[:'2021-95',:].dropna()

df=df.merge(df_meteo,left_index=True,right_index=True)

df

,Agriculteur,Production,RatioSep,RatioOct,Prix_matos,tmax_c 01,tmax_c 02,tmax_c 03,tmax_c 04,tmax_c 05,...,sunhour 43_n-1,sunhour 44_n-1,sunhour 45_n-1,sunhour 46_n-1,sunhour 47_n-1,sunhour 48_n-1,sunhour 49_n-1,sunhour 50_n-1,sunhour 51_n-1,sunhour 52_n-1
2010-1,2820.041792,239640.3,1.229426,1.255319,92.741667,2.0,1.0,1.0,1.0,1.0,...,7.328571,8.700000,6.328571,6.057143,8.214286,6.714286,5.528571,5.257143,6.428571,6.742857
2010-2,3563.698886,1485134.7,1.229426,1.255319,92.741667,2.0,1.0,2.0,0.0,1.0,...,6.142857,7.514286,3.657143,6.514286,6.842857,3.671429,3.900000,5.185714,6.914286,4.728571
2010-3,4276.654443,307450.0,1.229426,1.255319,92.741667,0.0,1.0,1.0,2.0,1.0,...,6.871429,8.700000,5.614286,7.014286,7.728571,6.157143,5.614286,4.414286,5.342857,6.642857
2010-4,1462.803043,6233.7,1.229426,1.255319,92.741667,1.0,2.0,1.0,1.0,1.0,...,7.600000,8.700000,8.471429,8.700000,7.971429,7.942857,7.700000,8.185714,8.342857,6.142857
2010-5,1406.337173,9753.9,1.229426,1.255319,92.741667,1.0,0.0,1.0,1.0,1.0,...,8.471429,8.700000,6.342857,8.457143,7.728571,7.700000,7.185714,8.085714,8.000000,6.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-91,422.282609,243243.0,1.104197,1.072703,89.891667,1.0,1.0,2.0,1.0,1.0,...,5.457143,5.400000,7.500000,6.028571,7.942857,7.800000,4.871429,3.371429,6.542857,3.914286
2021-93,80.734902,2739.0,1.104197,1.072703,89.891667,4.0,2.0,6.0,4.0,2.0,...,7.800000,7.542857,7.528571,7.514286,8.700000,7.728571,7.000000,6.485714,4.914286,7.442857
2021-94,65.237675,3483.0,1.104197,1.072703,89.891667,2.0,0.0,2.0,1.0,1.0,...,5.942857,5.400000,7.742857,6.257143,7.185714,7.285714,4.871429,3.357143,6.800000,3.900000
2021-95,213.709486,198121.0,1.104197,1.072703,89.891667,3.0,0.0,2.0,1.0,1.0,...,5.685714,5.171429,6.785714,5.271429,6.414286,7.285714,4.914286,4.171429,5.842857,3.100000
